In [1]:
import numpy as np
import pandas as pd
import torch

from model.GHATModel import GAT
from config import Config

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
conf = Config()

def build_adj():
    # connection = [
    # (1, 0),
    # (9, 0), (12, 0), 
    # (8, 9), (8, 12), (5, 9), (11, 12), 
    # (4, 5), (4, 8), (7, 8), (7, 11), (10, 11),
    # (3, 4), (3, 7), (6, 7), (6, 10), (2, 3), (2, 6)]
    
    # 无向图
    connection = [
        (1, 0), (0, 1),
        (9, 0), (12, 0), (0, 9), (0, 12),
        (8, 9), (8, 12), (5, 9), (11, 12), (9, 8), (12, 8), (9, 5), (12, 11),
        (4, 5), (4, 8), (7, 8), (7, 11), (10, 11), (5, 4), (8, 4), (8, 7), (11, 7), (11, 10),
        (3, 4), (3, 7), (6, 7), (6, 10), (2, 3), (2, 6), (4, 3), (7, 3), (7, 6), (10, 6), (3, 2), (6, 2)
        ]
    adj_matrix = torch.zeros(13, 13).float()
    for source, target in connection:
        adj_matrix[source][target] = 1
    return adj_matrix

In [2]:
input_data = np.load('./data/volume/0308/Input/000046_3_3_inputs.npy', allow_pickle= True)
input_data = np.array([np.array(item) for item in input_data[0]])
input_data = torch.from_numpy(input_data).to(torch.float32)
input_data = input_data.view(1, 1, 13, 9).to(device)
input_data.shape

torch.Size([1, 1, 13, 9])

In [3]:
model = GAT(n_feat= len(conf.train_features), n_hid= conf.n_hid, out_features= len(conf.pred_features), 
            pred_length= conf.pred_length, n_heads= conf.n_head)
state_dict = torch.load('./saved_models/model_epoch_64.pt')
model.load_state_dict(state_dict)
model.eval()
model(input_data, build_adj())[-1]

RuntimeError: Error(s) in loading state_dict for GAT:
	Unexpected key(s) in state_dict: "mlp.1.weight", "mlp.1.bias", "mlp.4.weight", "mlp.4.bias". 